In [ ]:
!pip install streamlit pyngrok


In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("Token")

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import plotly.express as px

df = pd.read_csv("/content/merged_df.csv", parse_dates=['published_dt_only'])

df['Daily_Return'] = df['Close_AAPL'].pct_change()

df['Volatility'] = df['High_AAPL'] - df['Low_AAPL']

st.title("Apple Stock & News Sentiment Dashboard")

st.sidebar.header("Filters")
start_date = st.sidebar.date_input("Start date", df['published_dt_only'].min())
end_date = st.sidebar.date_input("End date", df['published_dt_only'].max())

filtered_df = df[(df['published_dt_only'] >= pd.to_datetime(start_date)) &
                 (df['published_dt_only'] <= pd.to_datetime(end_date))]

fig_price = px.line(filtered_df, x='published_dt_only', y='Close_AAPL',
                    title="Apple Closing Price Over Time")
st.plotly_chart(fig_price)

fig_sentiment = px.line(filtered_df, x='published_dt_only', y='sentiment_score',
                        title="News Sentiment Over Time", color_discrete_sequence=["orange"])
st.plotly_chart(fig_sentiment)

fig_vol = px.scatter(filtered_df, x='Volume_AAPL', y='Volatility',
                     size='Volatility', color='Daily_Return',
                     hover_data=['Close_AAPL'],
                     title="Trading Volume vs Volatility")
st.plotly_chart(fig_vol)

fig_dist = px.histogram(filtered_df, x='sentiment',
                        title="Sentiment Distribution of News Articles",
                        color='sentiment')
st.plotly_chart(fig_dist)

source_sentiment = filtered_df.groupby('source')['sentiment_score'].mean().reset_index()
fig_sources = px.bar(source_sentiment, x='source', y='sentiment_score',
                     title="Average Sentiment by News Source")
st.plotly_chart(fig_sources)

st.subheader("Data Preview")
st.dataframe(filtered_df.head(10))


In [ ]:
from pyngrok import ngrok
import subprocess

ngrok.kill()

process = subprocess.Popen(["streamlit", "run", "app.py"])

public_url = ngrok.connect(8501)
print(" Public URL:", public_url.public_url)
